# RRNN Análisis de microarrays

__Objetivos de la práctica__

- Utilizar datos de microarrays para agrupar genes en función de sus patrones de expresión.
- Comparar los grupos obtenidos con los diferentes tipos de cáncer anotados.

El __conjunto de datos NCI60__ contiene información genética de __64 líneas celulares cancerígenas__. Para cada una de ellas, se ha cuantificado la expresión de __6830 genes__ mediante __tecnología microarray__. Se conoce el __tipo de cáncer__ (histopatología) al que pertenece cada línea celular y se puede utilizar esta información para evaluar si el método de clustering (agglomerative hierarchical clustering) es capaz de agrupar correctamente las líneas empleando los niveles de expresión génica.

In [1]:
import matplotlib.pyplot as plt 
%matplotlib inline
import pandas as pd
import numpy as np
from skimage import io
from IPython import display

## Carga de Datos

Descargar el archivo que se encuentra en Moodle con el nombre __nci_data.csv__ y situarlo en una carpeta de nombre __data__ en el path donde se encuentre el cuaderno Jupyter.

In [2]:
df = pd.read_csv('data/nci_data.csv')
df.head()

,Gene,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s55,s56,s57,s58,s59,s60,s61,s62,s63,s64
0,g1,0.300,0.679961,0.940,0.280,0.485,0.310,-0.830,-0.190,0.460,...,0.010,-0.620,-0.380,0.04998,0.650,-0.030,-0.270,0.210,-0.050,0.350
1,g2,1.180,1.289961,-0.040,-0.310,-0.465,-0.030,0.000,-0.870,0.000,...,-1.280,-0.130,0.000,-0.72002,0.640,-0.480,0.630,-0.620,0.140,-0.270
2,g3,0.550,0.169961,-0.170,0.680,0.395,-0.100,0.130,-0.450,1.150,...,-0.770,0.200,-0.060,0.41998,0.150,0.070,-0.100,-0.150,-0.090,0.020
3,g4,1.140,0.379961,-0.040,-0.810,0.905,-0.460,-1.630,0.080,-1.400,...,0.940,-1.410,0.800,0.92998,-1.970,-0.700,1.100,-1.330,-1.260,-1.230
4,g5,-0.265,0.464961,-0.605,0.625,0.200,-0.205,0.075,0.005,-0.005,...,-0.015,1.585,-0.115,-0.09502,-0.065,-0.195,1.045,0.045,0.045,-0.715


__Los 64 tipos de cancer__ se encuentra en el archivo __nci_label.txt__ también situado en Moodle y se descarga a la misma carpeta __data__

In [3]:
df_lbl = pd.read_csv('data/nci_label.txt', header=None, names=["Cell"])
df_lbl.head()
y = df_lbl.values
np.size(y), np.unique(y)

(64,
 array(['BREAST', 'CNS', 'COLON', 'K562A-repro', 'K562B-repro', 'LEUKEMIA',
        'MCF7A-repro', 'MCF7D-repro', 'MELANOMA', 'NSCLC', 'OVARIAN',
        'PROSTATE', 'RENAL', 'UNKNOWN'], dtype=object))

__La estructura del fichero nci_data.csv es__:

- Una columna que sirve para enumerar las 6830 expresiones genéticas y que no se utilizará.
- __64 columnas__ con los tipos de cancer.

Por tanto, se toman las 64 últimas columnas, por lo que es necesario transponer la matriz __X_trap__ para obtener el conjunto inicial de entrenamiento, que se denomina __X_ini__.

__X_ini__ ya tiene __64 filas__ por cada tipo histológico de cancer con __3680 características__, siendo cada uno de ellos la expresión de un gen.

In [4]:
X_trap = df.values[:,1:65]
X_ini = np.transpose(X_trap)
X_ini.shape

(64, 6830)

__Estandarización de los datos de entrada__

Para poder probar con datos estandarizados o no estandarizados en la siguiente celda tenemos las dos opciones de forma que en la matriz __X__ con la que se va a trabajar se podrá volcar el dato estandarizado o no, según se comente o descomente la última línea de código

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X_ini)
#X = X_ini

## División en Train y Test

Debido a la baja población no es posible hacer la división, pues quedarían clases con 1 sólo elemento. Se emplea todo __X__ e __y__

## Entrenamiento del Perceptron

Primero se echa un vistazo a las dimensiones del conjunto de entrada y a la cantidad de etiquetas de la variable objetivo

In [6]:
X.shape, np.size(np.unique(y))

((64, 6830), 14)

__Elegir el número de neuronas de la capa de entrada__

El número de neuronas de entrada es 6830 y las de salida son 14

In [7]:
i=6830
o=14
h=2*(i+o)/3
int(h), h

(4562, 4562.666666666667)

__Por compatibilidad y evitar Warnings se convierte la lista de tipos de cancer de Y en array__

In [8]:
y_array = np.ravel(y)

__Se adopta una capa oculta con las neuronas calculadas anteriormente__

Mostrar la exactitud de todo el conjunto de entrenamiento

In [10]:
from sklearn.neural_network import MLPClassifier
## Inicialización y creación del objeto clasificador
mlp = MLPClassifier(hidden_layer_sizes=(4562), max_iter=10,activation = 'logistic',solver='lbfgs',random_state=1, verbose=True)
mlp.fit(X, y_array)
print("Exactitud del conjunto de entrenamiento: {:.3f}".format(mlp.score(X, y_array)))

Exactitud del conjunto de entrenamiento: 1.000
